In [10]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df_ce_part1 = pd.read_csv("/content/drive/MyDrive/Thesis/CE Part1/RCS_250410_1612_5S2P_50CE_Run1_1.csv")

In [4]:
df_ce_part1.head()

,Time,V1,V2,V3,V4,V5,IApp,DV1,DV2,DV3,DV4,DV5,DT,FaultIN
0,1.00,7.309181,7.304908,7.303687,7.309791,7.310573,-0.023823,0.004273,0.001221,-0.006104,-0.000782,0.001392,0.148080,0
1,1.05,7.309181,7.304908,7.303687,7.309791,7.310841,-0.023823,0.004273,0.001221,-0.006104,-0.001050,0.001660,0.148080,0
2,1.10,7.309181,7.304908,7.303687,7.309791,7.310573,-0.023823,0.004273,0.001221,-0.006104,-0.000782,0.001392,0.148080,0
3,1.15,7.309181,7.304908,7.303687,7.309791,7.310402,-0.023823,0.004273,0.001221,-0.006104,-0.000611,0.001221,0.148080,0
4,1.20,7.309181,7.304908,7.303687,7.309791,7.310402,-0.023823,0.004273,0.001221,-0.006104,-0.000611,0.001221,0.149639,0


In [6]:
df_ce_part1.FaultIN.value_counts()

,count
FaultIN,
0,61772
1,12366


In [7]:
len(df_ce_part1)

74138

In [11]:
corr_matrix = df_ce_part1.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    labels=dict(x="Variable", y="Variable", color="Correlation"),
    title="Correlation Heatmap"
)

fig.update_layout(
    xaxis_tickangle=45,
    height=700,
    width=1200
)

fig.show()

In [17]:
df_ce_part1["Time_30sec"] = df_ce_part1["Time"] // 15

df_ce_part1_30sec_median = df_ce_part1.groupby("Time_30sec").median().reset_index()
df_ce_part1_30sec_max = df_ce_part1.groupby("Time_30sec").max().reset_index()

In [19]:
variables_to_plot = [col for col in df_ce_part1_30sec_median.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_ce_part1_30sec_median["Time"],
        y=df_ce_part1_30sec_median[var],
        mode='lines',
        name=var
    ))

fault_times = df_ce_part1_30sec_median[df_ce_part1_30sec_median["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        # annotation_text="Fault",
        # annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 15s, median",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()


In [20]:
variables_to_plot = [col for col in df_ce_part1_30sec_max.columns if col not in ["FaultIN", "Time"]]

fig = go.Figure()

for var in variables_to_plot:
    fig.add_trace(go.Scatter(
        x=df_ce_part1_30sec_max["Time"],
        y=df_ce_part1_30sec_max[var],
        mode='lines',
        name=var
    ))

fault_times = df_ce_part1_30sec_max[df_ce_part1_30sec_max["FaultIN"] == 1]["Time"]

for ft in fault_times:
    fig.add_vline(
        x=ft,
        line=dict(color="red", width=1, dash="dash"),
        # annotation_text="Fault",
        # annotation_position="top right"
    )

fig.update_layout(
    title="Variables Over Time with Fault Markers - 15s, max",
    xaxis_title="Time (seconds)",
    yaxis_title="Value",
    legend=dict(orientation="h"),
    height=600
)

fig.show()

In [21]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import plotly.graph_objects as go

# Shannon entropy function
def shannon_entropy(series, bins=100):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

# Use your median-aggregated DataFrame
df = df_ce_part1.copy()

# Entropy calculation
window_size = 100
entropy_values = [
    shannon_entropy(df["DT"].iloc[i : i + window_size])
    for i in range(len(df) - window_size + 1)
]

# Align time to match entropy window start
time_vals = df["Time"].iloc[: len(entropy_values)]

# Plotly plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_vals,
    y=entropy_values,
    mode="lines",
    name="Shannon Entropy (DT)",
    line=dict(color="red")
))

fig.update_layout(
    title="Shannon Entropy of DT Over Time",
    xaxis_title="Time (seconds)",
    yaxis_title="Entropy",
    height=500
)

fig.show()
